# Formulisation

Let i, j, k, t be order i, carrier j, plant k, day t.<br>
Let $X(i,j,k,t)$ be the amount of order i **SENT** on carrier j to plant k on day t <br>
>if it takes 2 days(tpt_day_cnt[j] = 2) to deliver n goods from i to k using j, then if we send an order of n goods on day 0 and send no order on day 1, we have X(i, j, k, 0) = n and X(i, j, k, 1) = 0, tho the order is still in-transport on day 1.<br>

Let $S_i$ be the weight[i]<br>
Let $R_j$ be rate[j]; let $D_j$ be tpt_day_cnt[j]; let $W_j$ be max_weight[j] <br>
Let $C_k$ be the capacity[k]<br>


Thus,<br>
$\sum_{t=0}^T X(i, j, k, t)$ is the **total** amount of shipment of order i to plant k using carrier j;<br>
$X(i, j, k, t-D_j)$ is the amount of shipment **delivered** on day t of order i to plant k using carrier j;<br>
$\sum_{d = max(0, t-D_j)}^{t} X(i, j, k, d)$ is the amount of shipment **in-transport** at time t of order i to plant k using carrier j

**Objective**: <br>
Minimise $ = \sum_{(i, j, k, t)} (R_j+0.01*(t+D_j))* X(i, j, k, t)$


**Constraints**:<br>
For each i, $\sum_{(j, k, t)} X(i, j, k, t) = S_i$<br>
For each i, j, k, t, $\sum_{d = max(0, t-D_j)}^{t} X(i, j, k, d)<=W_j$<br>
For each k and t, $\sum_{(i, j)}X(i, j, k, t-D_j)<=C_k$<br>

# Code

In [1]:
%pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 27.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/My Drive/Spring_2024/IEOR4004/Projects/Project_4

/content/drive/My Drive/Spring_2024/IEOR4004/Projects/Project_4


In [4]:
from gurobipy import GRB
import gurobipy as gp
import numpy as np
import pandas as pd
import sys
from tqdm import tqdm
from collections import defaultdict
from copy import deepcopy

In [5]:
class lp():
    def __init__(self, gp_params = None, plant_has_capacity = True):
        self.carrier_to_plant, self.plant_to_carrier, self.plant_capacity, self.carrier_weight, self.carrier_rate, self.carrier_day, self.order_weight = self.generate_dict()
        self.plant_has_capacity = plant_has_capacity
        if gp_params:
            self.env = gp.Env(params=gp_params)
        else:
            self.env = None

    def generate_dict(self):
        print('\n------Generating Dicts------')
        orderlist = pd.read_excel('project4.xlsx', sheet_name = 'OrderList', usecols="A:C").dropna()
        freightrates = pd.read_excel('project4.xlsx', sheet_name = 'FreightRates', index_col=0)
        plantlocation = pd.read_excel('project4.xlsx', sheet_name = 'Plant location')
        plantcapacity = pd.read_excel('project4.xlsx', sheet_name='Plant Capacity')

        carrier_to_plant = defaultdict(list)
        for j in freightrates.index:
            port_j = freightrates['destination port'].loc[j]
            plant_js = plantlocation['Plant Code'].loc[plantlocation['Port'] == port_j].tolist()
            for plant in plant_js:
                carrier_to_plant[j].append(int(plant[-2:]))

        plant_to_carrier = defaultdict(list)
        for k in range(len(plantlocation)):
            plant = plantlocation['Plant Code'].loc[k]
            plant_code = int(plant[-2:])
            port = plantlocation['Port'].loc[k]
            carrier_ks = freightrates.index[freightrates['destination port'] == port].tolist()
            plant_to_carrier[plant_code].extend(carrier_ks)

        plant_capacity = {int(ID[-2:]): cap for ID, cap in zip(plantcapacity['Plant ID'], plantcapacity['Daily Capacity (1e6)'])}

        carrier_weight = defaultdict(float)
        for j in freightrates.index:
            carrier_weight[j] = freightrates.loc[j]['max weight']

        carrier_rate = defaultdict(float)
        for j in freightrates.index:
            carrier_rate[j] = freightrates.loc[j]['rate']

        carrier_day = defaultdict(int)
        for j in freightrates.index:
            carrier_day[j] = freightrates.loc[j]['tpt_day_cnt']

        order_weight = defaultdict(float)
        for i in orderlist.index:
            order_weight[i] = orderlist.loc[i]['Weight']

        return carrier_to_plant, plant_to_carrier, plant_capacity, carrier_weight, carrier_rate, carrier_day, order_weight

    def lp(self):
        if self.plant_has_capacity:
            return self.lp_cap()
        else:
            return self.lp_no_cap()

    def lp_cap(self):
        ### TOOOOO LARGE TO RUN ###
        optimal_t = 200
        optimal_cost = float('inf')
        optimal_model = None
        optimal_xvar = {}

        for t in range(200):
            print(f'---------Creating Model When T = {t}---------')

            # Call lp_fixed_days method to set up the model
            model, xvar, total_cost = self.lp_cap_fixed_days(t)

            if model.status == GRB.INFEASIBLE:
                print(f'---------Failed at T == {t}---------')
                continue
            if model.status == GRB.OPTIMAL:
                if total_cost.x < self.optimal_cost:
                    print(f'---------Total Cost = {total_cost.x}---------')
                    self.optimal_t = t
                    self.optimal_cost = total_cost.x
                    self.optimal_model = model
                    self.optimal_xvar = xvar
                else:
                    print(f'---------Not Optimal---------')

        return self.optimal_model, self.optimal_xvar, self.optimal_cost, self.optimal_t

    def lp_cap_fixed_days(self, t_range):
        if self.env:
            m = gp.Model(f'freights_{t_range}', env = self.env)
        else:
            m = gp.Model(f'freights_{t_range}')

        print('****** Creating Variable List ******')
        xvar = {}
        for i in tqdm(self.order_weight.keys()):
                order_w = self.order_weight[i]
                for j in self.carrier_weight.keys():
                    carrier_w = self.carrier_weight[j]
                    ks = self.carrier_to_plant[j]
                    for k in ks:
                        for t in range(t_range+1):
                            x[i,j,k,t] = m.addVar(lb = 0.0, ub = max(order_w, carrier_w), name = f'X_{i},{j},{k},{t}')
        print('****** Setting Objective ******')
        total_cost = m.addVar(name='total_cost')
        m.setObjective(total_cost, GRB.MINIMIZE)
        m.addConstr(total_cost == sum((self.carrier_rate[j]+0.01*(t+self.carrier_rate[j]))*xvar[i,j,k,t] for i, j, k, t in xvar.keys()), name = 'Total_Cost')

        # First Constraint: Weights of each order
        print('****** Adding First Constraint ******')
        for i in tqdm(self.order_weight.keys()):
            S_i = self.orderlist.loc[i, 'Weight']
            m.addConstr(sum(xvar[i, j, k, t]
                            for j in self.carrier_weight.keys() for k in self.carrier_to_plant[j] for t in range(t_range+1)) == S_i, name = f'Order_{i}')

        # Second Constraint: Carrier Max Weight
        print('****** Adding Second Constraint ******')
        for i in tqdm(self.order_weight.keys()):
            for j in self.carrier_weight.keys():
                W_j = self.carrier_weight[j]
                D_j = self.carrier_day[j]
                for k in self.carrier_to_plant[j]:
                    expr = sum(xvar[i,j,k,t] for t in range(0, D_j+1))
                    m.addConstr(expr <= W_j, name = f'Max_Weight_{i}{j}{k}{D_j}')
                    for t in range(D_j+1, t_range + 1):
                        expr = expr - xvar[i,j,k,t-D_j-1] + xvar[i,j,k,t]
                        m.addConstr(expr <= W_j, name = f'Max_Weight_{i}{j}{k}{t}')

        if self.plant_has_capacity:
            # Third Constraint: Plant Capacity
            print('****** Adding Third Constraint ******')
            for k in tqdm(self.plant_capacity.keys()):
                C_k = self.plant_capacity[k]*1e6
                for t in range(t_range + 1):
                    expr = sum(xvar[i, j, k, (t-self.freightrates.loc[j, 'tpt_day_cnt'])] if (t-self.freightrates.loc[j, 'tpt_day_cnt'])>=0 else 0
                                    for i in self.order_weight.keys() for j in self.plant_to_carrier[k])
                    m.addConstr(expr <= C_k, name = f'Plant_{k}_at_{t}')

        try:
            m.update()
            print('****** Optimising ******')
            m.setParam('NodefileStart', 10)
            m.setParam('Threads', 4)
            m.optimize()
        except gp.GurobiError as e:
            print(f'Error code {e.errno}: {e}', file=sys.stderr)
        except AttributeError:
            print('Encountered an attribute error', file=sys.stderr)

        return m, xvar, total_cost

    def lp_no_cap(self):
        total_cost = 0
        max_t = 0
        with open('logistics_no_cap.dat', 'w') as file:
            file.write(f"{len(self.order_weight)} {len(self.carrier_day)} {len(self.plant_capacity)} 100\n")
            # Iterate over each order
            for i in tqdm(range(len(self.order_weight))):
                optimal_model, optimal_ivar, optimal_cost, optimal_t = self.lp_no_cap_i(i)
                total_cost += optimal_cost
                max_t = max(max_t, optimal_t)
                for j, k, t in optimal_ivar.keys():
                    if optimal_ivar[j, k, t].x > 1e-6:
                    # Write the data to the file in the desired format
                        file.write(f"{i} {j} {k} {t} {optimal_ivar[j, k, t].x}\n")
            # Write END to indicate the end of the data
            file.write("END\n")
        return total_cost, max_t

    def lp_no_cap_i(self, i):
        optimal_t = 200
        optimal_cost = float('inf')
        optimal_model = None
        optimal_ivar = {}
        if self.order_weight[i] < 1e5:
            t_range = [0]
        elif self.order_weight[i] < 2e7:
            t_range = np.linspace(0, 9, 10, dtype = int)
        else:
            t_range = np.logspace(1, 2, 10, dtype = int)

        for t in t_range:
            model, ivar, icost = self.lp_no_capacity_i_fixed_day(i, t)

            if model.status == GRB.INFEASIBLE:
                continue
            if model.status == GRB.OPTIMAL:
                if icost.x < optimal_cost:
                    optimal_t = t
                    optimal_cost = icost.x
                    optimal_model = model
                    optimal_ivar = ivar

        return optimal_model, optimal_ivar, optimal_cost, optimal_t

    def lp_no_capacity_i_fixed_day(self, i, t_range):
        # Build Model
        if self.env:
            m = gp.Model(f'freights_{t_range}', env = self.env)
        else:
            m = gp.Model(f'freights_{t_range}')
        m.setParam('LogToConsole', 0)
        xvar = {}
        for j in self.carrier_day.keys():
            max_weight = self.carrier_weight[j]
            for k in self.carrier_to_plant[j]:
                for t in range(0, t_range+1, self.carrier_day[j]+1):
                    xvar[j, k, t] = m.addVar(ub=max_weight, name=f'x_{i},{j},{k},{t}')
        i_cost = m.addVar(name=f'Order_{i}_cost')
        m.setObjective(i_cost, GRB.MINIMIZE)
        expr = gp.quicksum((self.carrier_rate[j]+0.01*(t+self.carrier_day[j]))*xvar[j,k,t]
                           for j, k, t in xvar.keys())
        m.addConstr(i_cost == expr, name = f'Cost_{i}')
        m.addConstr(gp.quicksum(xvar[j, k, t]
                            for j, k, t in xvar.keys()) == self.order_weight[i], name = f'OrderWeight_{i}')
        try:
            m.update()
            m.setParam('NodefileStart', 10)
            m.setParam('Threads', 4)
            m.optimize()
        except gp.GurobiError as e:
            print(f'Error code {e.errno}: {e}', file=sys.stderr)
        except AttributeError:
            print('Encountered an attribute error', file=sys.stderr)

        return m, xvar, i_cost

In [ ]:
# Type In Your Own gurobi.lic
params = {
    "WLSACCESSID": 'YOUR WLS ID',
    "WLSSECRET": 'YOUR WLS SECRET',
    "LICENSEID": int('YOUR LICENSE'),
}
lp_instance = lp(gp_params = params, plant_has_capacity = False)
cost, t = lp_instance.lp()
print(f'\nTotal Time Needed: {t}')
print(f'Total Cost: {cost}')

In [8]:
f = open('logistics_no_cap.dat',"r")
lines = f.readlines();
f.close()
max_sent = 0
max_delivered = 0
total_cost = 0.0
for i in tqdm(range(1, len(lines)-1)):
    thisline = lines[i].split()
    t_sent = int(thisline[3])
    t_delivered = t_sent + lp_instance.carrier_day[int(thisline[1])]
    max_sent = max(max_sent, t_sent)
    max_delivered = max(max_delivered, t_delivered)
    total_cost += float(thisline[4])

print(f'\nThe last goods sent on day {max_sent}')
print(f'The last goods delivered on day {max_delivered}')
print(f'The total cost is {total_cost}')

100%|██████████| 252270/252270 [00:00<00:00, 453494.92it/s]


The last goods sent on day 72
The last goods delivered on day 74
The total cost is 3960290083.400903


In [6]:
lp_instance = lp(plant_has_capacity = False)
f = open('logistics_no_cap.dat',"r")
lines = f.readlines();
f.close()
plant_storage = {}
for i in range(1, 20):
    plant_storage[i] = [0]*75
for i in tqdm(range(1, len(lines)-1)):
    thisline = lines[i].split()
    k = int(thisline[2])
    t_sent = int(thisline[3])
    x = float(thisline[4])
    t_delivered = t_sent + lp_instance.carrier_day[int(thisline[1])]
    plant_storage[k][t_delivered] += x
df = pd.DataFrame(plant_storage)
df.to_csv('plant_storage_no_cap.csv')


------Generating Dicts------


100%|██████████| 252270/252270 [00:00<00:00, 511565.28it/s]
